# Case study: state machine

This case study illustrates the use of the staircase package for analysing data derived from a [state machine](https://en.wikipedia.org/wiki/State_diagram). States are either active or not active, and transitions between states are triggered by events.  It is these events (and the time at which they occur) which comprise the data.

The data used is this case study is synthetic and fictional.  Both data and the notebook for this tutorial can be obtained from the [github site](https://github.com/venaturum/staircase/tree/master/docs/examples).

The state machine diagram is pictured below.  Note that state A is the initial state (at time = 0).  Note that the transitions between states A and B, and the transitions between C and D, share the same labels.  This complicates the analysis, however we can still derive the results with some step function arithmetic! 

![State Chart](./img/state_chart.png)


In [ ]:
import pandas as pd
import staircase as sc

We begin by importing the event data into a pandas.DataFrame instance. Each row corresponds to an event.  The first column identifies the event type, while the second column gives the time (as some unspecified unit of time past 0) at which the event occurred.

In [ ]:
data = pd.read_csv('./data/state_machine.csv')
data

If the transition from A to B, and B to A, were instead labelled *y* and *z* respectively, then analysing with staircase would be straightforward.  For example, a binary-valued step function representing when state B is active could be obtained with the following code:

![code](./img/state_B_code.png)

We will take a similar approach but we will need to appeal to compound states - a state composed of multiple substates.  We will denote the compound state AB as meaning in state A or B, and likewise for compound states CD and BD.

Compound state AB becomes active whenever event *q* occurs, and becomes inactive when event *p* occurs.  Also note that we start in AB at time = 0.  We can create a step function for AB with the following code:

In [ ]:
AB = sc.Stairs().layer(0).layer(data[data.event == "q"].time, data[data.event == "p"].time)
AB.plot()

Note the first call to the layer method, increases the value of the step function from 0 to 1, at time 0.  It is also important to know that there is no pairing between *q* and *p* events.  In fact we start in state A, and finish in state C, and so there aren't even an equal number of *p* and *q* events in our data (see value_counts result below).  This is fine however, with the layer method simply interpreting the *start* parameter as times at which the step function increases by 1, and the *end* parameter as times at which the step function decreases by 1.

In [ ]:
data.event.value_counts()

We can also derive step functions for compound states CD and BD like so:

In [ ]:
CD = sc.Stairs(0, data[data.event == "p"].time, data[data.event == "q"].time)
BD = sc.Stairs(0, data[data.event == "w"].time, data[data.event == "x"].time)

Note that unlike the singular states, it is possible to be in more than one compound state at any time.  In fact it this result that our approach relies upon.  If compound state AB is active, and compound state BD is active, then state B must be active (and the converse is true also).  Therefore we can create a step function for state B by multiplying the step functions for AB and BD together.

In [ ]:
B = AB*BD
B.plot()

We can then obtain a step function for state A by using the fact that if state A is active, then compound state AB is active and state B is not (and the converse is true.  This is equivalent to subtracting the step function for state B from the step function for compound state AB:

In [ ]:
A = AB - B
A.plot()

The same approach can be taken to derive step functions for states D and C:

In [ ]:
D = CD*BD
C = CD - D

If we add the step functions for these singular states together then we obtain a step function for the compound state composed of all singular states.  This step function should be zero valued and negative infinity and transitioning to a value of 1 at time = 0.  We can verify this by plotting the result, remembering that the infinite length segments of a step function are not plotted:

In [ ]:
(A + B + C + D).plot()

Now that we have step functions for the singular states we can answer questions such as "what fraction of time during the first 200 time units was spent in time A?": (38.1%)

In [ ]:
A.mean(0,200)